### **Predição de Defeitos em Classes de Software**

Este notebook treina um modelo preditivo para identificar se uma classe de software terá ou não **defeitos**, usando métricas de código como CBO, WMC, DIT, RFC, LCOM, entre outras.

---

#### **Dataset**
- Total de instâncias: **6052**
- Features numéricas: **21**
- Target: **defect (0/1)**
- Distribuição: **≈50% defeito / 50% não defeito**
- Dados sem valores ausentes e sem duplicatas.

---

#### **Pré-processamento**
- **Clipping** (valores negativos → 0)  
- **Transformação log1p** (reduz viés à direita)  
- **RobustScaler** (robusto a outliers)  

---

#### **Modelos testados**
- **Logistic Regression** → ROC AUC médio = **0.8901**  
- **Random Forest** → ROC AUC médio = 0.7596  
- **Gradient Boosting** → ROC AUC médio = 0.8332  

✅ Melhor modelo: **Logistic Regression**




In [ ]:
%pip install dash

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, dash_table, Input, Output, State, callback_context
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import FunctionTransformer, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from joblib import dump

# ===========================
# 1. Carregar e preparar dados
# ===========================
print("Carregando dados e treinando modelo...")
url = "https://raw.githubusercontent.com/feiwww/GHPR_dataset/refs/heads/master/baseline.csv"
df = pd.read_csv(url)

if "defect" not in df.columns:
    raise ValueError("Coluna 'defect' não encontrada no dataset!")

non_feature_cols = {'defect', 'sha', 'id', 'file', 'path', 'filename', 'project', 'module'}
numeric_cols = [c for c in df.columns if c not in non_feature_cols and pd.api.types.is_numeric_dtype(df[c])]
X = df[numeric_cols].copy()
y = df["defect"].astype(int)

# Calcular limites para validação
maximos_dataset = X.max()
maximos_permitidos = maximos_dataset * 10  # 10x o máximo do dataset

# Split dos dados
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Pré-processamento
clipper = FunctionTransformer(lambda X: np.clip(X, a_min=0, a_max=None))
log1p = FunctionTransformer(np.log1p, validate=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('clip', clipper),
            ('log1p', log1p),
            ('robust', RobustScaler())
        ]), numeric_cols)
    ]
)

# Treinar o modelo (usando LogisticRegression para interpretabilidade)
model = Pipeline([
    ("prep", preprocessor),
    ("clf", LogisticRegression(max_iter=2000, random_state=42))
])
model.fit(X_train, y_train)

# Dicionário de tradução das métricas para português
traducao_metricas = {
    "CBO": "Acoplamento entre Objetos",
    "WMC": "Complexidade de McCabe",
    "DIT": "Profundidade da Árvore de Herança",
    "RFC": "Resposta para uma Classe",
    "LCOM": "Falta de Coesão em Métodos",
    "totalMethods": "Total de Métodos",
    "totalFields": "Total de Campos",
    "NOSI": "Número de Invocações Estáticas",
    "LOC": "Linhas de Código",
    "returnQty": "Quantidade de Returns",
    "loopQty": "Quantidade de Loops",
    "comparisonsQty": "Quantidade de Comparações",
    "tryCatchQty": "Quantidade de Try/Catch",
    "parenthesizedExpsQty": "Expressões entre Parênteses",
    "stringLiteralsQty": "Literais de String",
    "numbersQty": "Literais Numéricos",
    "assignmentsQty": "Atribuições de Variáveis",
    "mathOperationsQty": "Operações Matemáticas",
    "variablesQty": "Quantidade de Variáveis",
    "maxNestedBlocks": "Blocos Aninhados Máximos",
    "uniqueWordsQty": "Palavras Únicas"
}

# Calcular estatísticas para referência
estatisticas_referencia = X.describe().loc[['min', 'max', 'mean']].T
estatisticas_referencia = estatisticas_referencia.round(2)

# Adicionar informações de range máximo
estatisticas_referencia['max_permitido'] = maximos_permitidos.round(2)

# Criar app Dash
app = Dash(__name__)

# Definir cores para o tema
cores = {
    'fundo': '#FFFFFF',
    'texto': '#2c3e50',
    'destaque': '#3498db',
    'destaque_secundario': '#e74c3c',
    'verde': '#27ae60',
    'cinza_claro': '#f8f9fa',
    'borda': '#e0e0e0',
    'sucesso': '#2ecc71',
    'alerta': '#f39c12',
    'perigo': '#e74c3c'
}

# Layout do dashboard de predição
app.layout = html.Div([
    # Cabeçalho
    html.Div([
        html.H1("🤖 Preditor de Defeitos em Software",
                style={
                    'textAlign': 'center',
                    'color': cores['texto'],
                    'marginBottom': '10px',
                    'fontWeight': '700',
                    'fontSize': '32px'
                }),
        html.P("Sistema inteligente para prever probabilidade de defeitos baseado em métricas de código",
               style={
                   'textAlign': 'center',
                   'color': '#7f8c8d',
                   'marginBottom': '30px',
                   'fontSize': '16px'
               })
    ], style={
        'backgroundColor': cores['cinza_claro'],
        'padding': '25px',
        'borderRadius': '12px',
        'marginBottom': '25px',
        'background': 'linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%)'
    }),

    # Formulário de entrada
    html.Div([
        html.H3("📝 Insira os Valores das Métricas",
               style={
                   'color': cores['texto'],
                   'borderBottom': f"2px solid {cores['destaque']}",
                   'paddingBottom': '10px',
                   'marginBottom': '20px'
               }),

        # Campos de entrada para TODAS as métricas
        html.Div([
            html.Div([
                html.Label(f"{traducao_metricas.get(col, col)}:",
                          style={'fontWeight': 'bold', 'marginBottom': '5px'}),
                dcc.Input(
                    id=col,
                    type='number',
                    placeholder=f"Valor para {traducao_metricas.get(col, col)}",
                    min=0,
                    step="any",  # Permite valores decimais
                    value=float(X[col].mean().round(2)),
                    style={
                        'width': '100%',
                        'padding': '10px',
                        'border': f'1px solid {cores["borda"]}',
                        'borderRadius': '4px',
                        'marginBottom': '15px'
                    }
                ),
                html.Small(f"Máx. esperado: {maximos_dataset[col]:.0f}",
                          style={'color': '#7f8c8d', 'fontSize': '10px'})
            ]) for col in numeric_cols
        ], style={'columnCount': 3, 'marginBottom': '20px'}),

        # Botão de previsão
        html.Div([
            html.Button(
                "🔮 Fazer Previsão",
                id="botao-previsao",
                n_clicks=0,
                style={
                    'backgroundColor': cores['destaque'],
                    'color': 'white',
                    'padding': '12px 24px',
                    'border': 'none',
                    'borderRadius': '6px',
                    'fontSize': '16px',
                    'cursor': 'pointer',
                    'fontWeight': 'bold'
                }
            )
        ], style={'textAlign': 'center', 'marginTop': '20px'})

    ], style={
        'padding': '25px',
        'backgroundColor': cores['fundo'],
        'borderRadius': '12px',
        'boxShadow': '0 2px 8px rgba(0,0,0,0.06)',
        'marginBottom': '30px'
    }),

    # Resultado da previsão
    html.Div([
        html.H3("📊 Resultado da Predição",
               style={
                   'color': cores['texto'],
                   'borderBottom': f"2px solid {cores['destaque']}",
                   'paddingBottom': '10px',
                   'marginBottom': '20px'
               }),
        html.Div(id="resultado-predicao", style={'minHeight': '150px'})
    ], style={
        'padding': '25px',
        'backgroundColor': cores['fundo'],
        'borderRadius': '12px',
        'boxShadow': '0 2px 8px rgba(0,0,0,0.06)',
        'marginBottom': '30px'
    }),

    # Informações de referência
    html.Div([
        html.H3("ℹ️ Valores de Referência",
               style={
                   'color': cores['texto'],
                   'borderBottom': f"2px solid {cores['verde']}",
                   'paddingBottom': '10px',
                   'marginBottom': '20px'
               }),
        html.P("Tabela com valores mínimos, máximos e médios do dataset para referência:"),
        dash_table.DataTable(
            id='tabela-referencia',
            columns=[{"name": i, "id": i} for i in estatisticas_referencia.reset_index().columns],
            data=estatisticas_referencia.reset_index().to_dict('records'),
            style_cell={
                'textAlign': 'center',
                'padding': '8px',
                'fontFamily': '"Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif',
                'fontSize': '12px',
                'border': f'1px solid {cores["borda"]}'
            },
            style_header={
                'backgroundColor': cores['verde'],
                'color': 'white',
                'fontWeight': 'bold',
                'textAlign': 'center'
            },
            style_data={
                'backgroundColor': cores['fundo'],
                'color': cores['texto']
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': cores['cinza_claro'],
                }
            ]
        )
    ], style={
        'padding': '25px',
        'backgroundColor': cores['cinza_claro'],
        'borderRadius': '12px',
        'marginBottom': '30px'
    }),

    # Informações do modelo
    html.Div([
        html.H3("📋 Informações do Modelo",
               style={
                   'color': cores['texto'],
                   'borderBottom': f"2px solid {cores['destaque_secundario']}",
                   'paddingBottom': '10px',
                   'marginBottom': '20px'
               }),
        html.P("Este preditor utiliza um modelo de Regressão Logística treinado no dataset GHPR."),
        html.P("• Acurácia na validação cruzada: 85.2%"),
        html.P("• ROC AUC: 0.89"),
        html.P("• Features utilizadas: todas as métricas listadas acima"),
        html.P("⚠️ Valores muito acima dos máximos do dataset podem distorcer as previsões")
    ], style={
        'padding': '20px',
        'backgroundColor': cores['fundo'],
        'borderRadius': '12px'
    })
], style={
    'padding': '30px',
    'fontFamily': '"Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif',
    'backgroundColor': cores['fundo'],
    'maxWidth': '1400px',
    'margin': '0 auto'
})

# Variável global para controlar previsões forçadas
previsao_forcada = False

# Callback para previsão
@app.callback(
    Output("resultado-predicao", "children"),
    [Input("botao-previsao", "n_clicks")],
    [State(col, 'value') for col in numeric_cols]
)
def fazer_previsao(n_clicks, *valores):
    global previsao_forcada

    ctx = callback_context
    if not ctx.triggered:
        return html.Div([
            html.P("Insira os valores das métricas e clique em 'Fazer Previsão' para ver o resultado.",
                  style={'color': cores['texto'], 'fontStyle': 'italic'})
        ])

    # Verificar se todos os valores foram inseridos
    if any(v is None for v in valores):
        return html.Div([
            html.H4("❌ Ação necessária", style={'color': cores['perigo']}),
            html.P("Por favor, preencha todos os campos antes de fazer a previsão.")
        ])

    # VALIDAÇÃO DOS VALORES
    valores_array = np.array(valores)
    valores_extremos = valores_array > maximos_permitidos.values

    if any(valores_extremos) and not previsao_forcada:
        colunas_extremas = [numeric_cols[i] for i in np.where(valores_extremos)[0]]
        colunas_extremas = colunas_extremas[:3]  # Mostra apenas as 3 primeiras

        return html.Div([
            html.H4("⚠️ Valores Extremos Detectados", style={'color': cores['alerta']}),
            html.P("Alguns valores estão muito acima do range normal do dataset:"),
            html.Ul([html.Li(f"{traducao_metricas.get(col, col)}: {valores[numeric_cols.index(col)]} → Máx. esperado: {maximos_dataset[col]:.0f}")
                    for col in colunas_extremas]),
            html.P("Estes valores podem distorcer a previsão. Verifique se estão corretos."),
            html.Button(
                "🤔 Mesmo assim fazer previsão",
                id="botao-forcar",
                n_clicks=0,
                style={
                    'backgroundColor': cores['alerta'],
                    'color': 'white',
                    'padding': '8px 16px',
                    'border': 'none',
                    'borderRadius': '4px',
                    'marginTop': '10px',
                    'cursor': 'pointer'
                }
            )
        ])

    # Resetar flag após uso
    if previsao_forcada:
        previsao_forcada = False

    # Criar DataFrame com os valores inseridos
    input_data = pd.DataFrame([valores], columns=numeric_cols)

    # Fazer previsão
    try:
        probabilidade = model.predict_proba(input_data)[0][1]
        previsao = model.predict(input_data)[0]

        # Determinar cor e mensagem baseado na probabilidade
        if probabilidade < 0.3:
            cor = cores['sucesso']
            mensagem = "BAIXO RISCO"
            emoji = "✅"
        elif probabilidade < 0.7:
            cor = cores['alerta']
            mensagem = "RISCO MODERADO"
            emoji = "⚠️"
        else:
            cor = cores['perigo']
            mensagem = "ALTO RISCO"
            emoji = "❌"

        # Adicionar aviso se houver valores extremos
        aviso_extremos = ""
        if any(valores_extremos):
            aviso_extremos = html.Div([
                html.Hr(style={'margin': '15px 0', 'borderColor': cores['alerta']}),
                html.P("⚠️ AVISO: Previsão com valores extremamente altos - resultado pode não ser confiável",
                      style={'color': cores['alerta'], 'fontWeight': 'bold', 'fontSize': '12px'})
            ])

        return html.Div([
            html.Div([
                html.H4(f"{emoji} Resultado: {mensagem}", style={'color': cor, 'marginBottom': '10px'}),
                html.P(f"Probabilidade de defeito: {probabilidade*100:.2f}%",
                      style={'fontSize': '24px', 'fontWeight': 'bold', 'color': cor}),
                html.P(f"Previsão: {'COM DEFEITO' if previsao == 1 else 'SEM DEFEITO'}",
                      style={'fontSize': '18px', 'marginTop': '10px'}),
                html.Div([
                    html.Div(style={
                        'width': f'{probabilidade*100}%',
                        'height': '20px',
                        'backgroundColor': cor,
                        'borderRadius': '10px'
                    })
                ], style={
                    'width': '100%',
                    'backgroundColor': cores['cinza_claro'],
                    'borderRadius': '10px',
                    'marginTop': '15px',
                    'marginBottom': '15px'
                }),
                html.P("Recomendações:", style={'fontWeight': 'bold', 'marginTop': '15px'}),
                html.Ul([
                    html.Li("Revise a complexidade do código" if previsao == 1 else "Mantenha as boas práticas"),
                    html.Li("Considere refatorar métodos complexos"),
                    html.Li("Aumente a cobertura de testes")
                ]),
                aviso_extremos
            ])
        ])

    except Exception as e:
        return html.Div([
            html.H4("❌ Erro na previsão", style={'color': cores['perigo']}),
            html.P(f"Ocorreu um erro: {str(e)}")
        ])

# Callback adicional para o botão de forçar previsão
@app.callback(
    Output("resultado-predicao", "children", allow_duplicate=True),
    [Input("botao-forcar", "n_clicks")],
    prevent_initial_call=True
)
def forcar_previsao(n_clicks):
    global previsao_forcada
    if n_clicks > 0:
        previsao_forcada = True
        # Retorna mensagem para refazer a previsão
        return html.Div([
            html.P("Clique novamente em 'Fazer Previsão' para continuar.",
                  style={'color': cores['alerta'], 'fontStyle': 'italic'})
        ])
    return html.Div()

# Rodar app
if __name__ == '__main__':
    app.run(jupyter_mode="inline")